In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def categorical_to_numerical(train, test):    
    train_ = train.copy()
    test_ = test.copy()  
    encoders_train = {}
    encoders_test = {} 
    for column in train.columns[:-1]:
        if train[column].dtype == np.object:
            label = LabelEncoder()
            encoders_train[column] = label.fit_transform(train[column])
            train_[column] = label.transform(train[column])
            
            label = LabelEncoder()
            encoders_test[column] = label.fit_transform(test[column])
            test_[column] = label.transform(test[column])
    train_.fillna(0,inplace=True)
    test_.fillna(0,inplace=True)   
    return train_, test_, encoders_train, encoders_test

In [4]:
def RMSE(true_y, preds):
    return np.sqrt(mean_squared_error(true_y, preds))
#rmse_scorer = make_scorer(RMSE, greater_is_better=False)

In [5]:
def get_file(test,predictions, file_name):
    sample = pd.read_csv('Sample_Submission_Tm9Lura.csv')
    sample['User_ID'] = test.User_ID.values
    sample['Product_ID'] = test.Product_ID.values
    sample['Purchase'] = predictions
    sample.to_csv(file_name, index=False)

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train_, test_, encoder_train, encoder_test = categorical_to_numerical(train, test)
X_train = train_.drop(['Purchase'],axis=1)
y_train = train_.Purchase
train_X, test_X,train_y, test_y = train_test_split(X_train, y_train,random_state=100, test_size = 0.2)

In [8]:
train.shape

(550068, 12)

In [25]:
NN_model = Sequential()

# Input Layer :
NN_model.add(Dense(100, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# Hidden Layers :
NN_model.add(Dense(50, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(50, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(50, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(50, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(50, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(50, kernel_initializer='normal',activation='relu'))
# Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 100)               1200      
_________________________________________________________________
dense_34 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_35 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_36 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_37 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_38 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_39 (Dense)             (None, 50)                2550      
__________

In [26]:
NN_model.fit(X_train, y_train, epochs=10, validation_split = 0.2)

Train on 440054 samples, validate on 110014 samples
Epoch 1/10
440054/440054 [==============================] - 103s 234us/step - loss: 25182543.4868 - mean_squared_error: 25182543.4868 - val_loss: 27153259.8293 - val_mean_squared_error: 27153259.8293
Epoch 2/10
440054/440054 [==============================] - 94s 213us/step - loss: 24820073.6456 - mean_squared_error: 24820073.6456 - val_loss: 26312300.6909 - val_mean_squared_error: 26312300.6909
Epoch 3/10
440054/440054 [==============================] - 86s 196us/step - loss: 24725804.4186 - mean_squared_error: 24725804.4186 - val_loss: 26186320.8552 - val_mean_squared_error: 26186320.8552
Epoch 4/10
440054/440054 [==============================] - 115s 261us/step - loss: 24618373.1978 - mean_squared_error: 24618373.1978 - val_loss: 26037641.1650 - val_mean_squared_error: 26037641.1650
Epoch 5/10
440054/440054 [==============================] - 82s 186us/step - loss: 24495898.9346 - mean_squared_error: 24495898.9346 - val_loss: 26363

In [27]:
RMSE(NN_model.predict(X_train),y_train)

4717.5280738993815